In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
import vcf

%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from pylab import plot, show, savefig, xlim, figure, hold, ylim, legend, boxplot, setp, axes
from itertools import compress
from pylab import MaxNLocator
import seaborn as sns; sns.set()
from matplotlib.colors import LogNorm
from matplotlib import gridspec
import ast
import itertools
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import fastcluster
from sklearn import cluster, datasets
import scipy.cluster.hierarchy as hier
from sklearn.cluster import KMeans
import time
import sys

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC

from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
import itertools
import pickle

import networkx as nx
import scipy

#for exporting to Adobe Illustrator
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

############################################################################################################################################################################################################################

# [1] Filter Sample Annotation

############################################################################################################################################################################################################################

### Import annotation files for samples

In [14]:
#annotation file for all isolates and corresponding tags
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/annotation CSV files/sample_fastq_path_names.csv' , sep = ',')
sample_annotation.loc[:, 'tag'] = ['S' + str(S_i) for S_i in list(sample_annotation.sample_num)]
sample_annotation.set_index('tag' , inplace = True , drop = True)

#each sub-directory has the pilon-outputed VCF file, lineage call and Qualimap report
parent_output_dir = '/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/JankyPipe/output/'

In [15]:
sample_annotation.head()

,sample_id,sample_num,fastq_files
tag,,,
S0,Undetermined,0,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S1,TC100810,1,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S2,TC94815,2,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S3,TC101527,3,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S4,LB95669,4,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...


### [1] Check to see which sample ran through pipeline successfully

In [16]:
good_sample_filter = []

for sample_tag in sample_annotation.index:
    
    VCF_file = parent_output_dir + sample_tag + '/pilon/' + sample_tag + '.vcf'
    
    #check to see if VCF file exists and that VCF file size is reasonable (> 100 MB)
    if os.path.exists(VCF_file) and (os.stat(VCF_file).st_size > 100000000):
            
        good_sample_filter.append(True)
            
    else:
        
        good_sample_filter.append(False)

#### FILTER: Had to *drop* samples below

In [17]:
sample_annotation[[not good_sample for good_sample in good_sample_filter]]

,sample_id,sample_num,fastq_files
tag,,,
S5,ERT102814,5,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...


In [18]:
sample_annotation = sample_annotation[good_sample_filter]

In [19]:
sample_annotation.head()

,sample_id,sample_num,fastq_files
tag,,,
S0,Undetermined,0,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S1,TC100810,1,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S2,TC94815,2,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S3,TC101527,3,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...
S4,LB95669,4,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...


### [2] Get coverage for each sample

In [20]:
#dictionary that stores the mean coverage for each isolate (that successfully ran through megapipe2.0) from QUALIMAP
mean_coverage_dict = {} #key: sample_ID , value: mean coverage

for sample_tag in sample_annotation.index:

    #QUALIMAP DATA
    ########################################################################################################################
    #look for qualimap output txt file that has mean coverage & mean read length
    qualimap_BAM_file_stats = parent_output_dir + sample_tag + '/QualiMap/' + 'genome_results.txt'

    #parse qualimap txt file and store the mean coverage for the BWA mapping (BAM file) & mean read length
    with open(qualimap_BAM_file_stats ,'r') as f:

        #iterate through lines in text file
        for stat_per_line in f:

            #find the mean coverage for mapping
            if 'mean coverageData' in stat_per_line:

                mean_coverage = float( stat_per_line.split('=')[-1][:-2].replace(',' , '') )
                mean_coverage_dict[sample_tag] = mean_coverage

                break #once we have mean coverage
    ########################################################################################################################
    
#convert to series
mean_coverage_series = pd.Series(mean_coverage_dict)

#append to sample annotation 
sample_annotation.loc[:, 'coverage'] = mean_coverage_series[sample_annotation.index]

In [21]:
sample_annotation

,sample_id,sample_num,fastq_files,coverage
tag,,,,
S0,Undetermined,0,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,25.5521
S1,TC100810,1,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,151.0760
S2,TC94815,2,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,60.5541
S3,TC101527,3,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,70.5301
S4,LB95669,4,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,82.3331
S6,MG90097,6,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,23.7142
S7,TC101848,7,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,92.6566
S8,SM94290,8,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,72.2888
S9,GLK101604,9,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,132.7089


#### FILTER: Had to *drop* samples below

In [22]:
sample_annotation[sample_annotation.coverage < 20]

,sample_id,sample_num,fastq_files,coverage
tag,,,,
S17,TN106697,17,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,0.0001


In [23]:
sample_annotation = sample_annotation[sample_annotation.coverage >= 20]

In [24]:
sample_annotation.head()

,sample_id,sample_num,fastq_files,coverage
tag,,,,
S0,Undetermined,0,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,25.5521
S1,TC100810,1,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,151.0760
S2,TC94815,2,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,60.5541
S3,TC101527,3,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,70.5301
S4,LB95669,4,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,82.3331


In [25]:
np.shape(sample_annotation)

(24, 4)

In [26]:
print list( sample_annotation.coverage.sort_values() )

[23.7142, 25.5521, 36.6669, 41.3522, 44.8859, 55.9323, 57.8258, 60.5541, 68.3426, 69.7764, 70.5301, 72.2888, 76.4243, 81.1897, 82.3331, 84.0541, 86.5909, 89.3929, 92.6566, 99.2839, 100.1381, 125.0654, 132.7089, 151.076]


### [3] Get lineage call for each sample

In [46]:
#dictionary that stores the global lineage call for each isolate
lineage_call_dict = {} #key: sample_ID , value: lineage_call

for sample_tag in sample_annotation.index:

    #fast-lineage-caller (from Luca)
    ########################################################################################################################
    #look for fast-lineage-caller output txt file that has the lineage call
    fast_lineage_caller_call = parent_output_dir + sample_tag + '/fast-lineage-caller/' + 'lineage_call.txt'

    #parse qualimap txt file and store the mean coverage for the BWA mapping (BAM file) & mean read length
    with open(fast_lineage_caller_call ,'r') as f:
        
        #iterate through lines in text file
        for lineage_call_line in f:
        
            lineage_call_i = lineage_call_line.split('lineage')[-1]
            
    lineage_call_dict[sample_tag] = lineage_call_i[:-1]
    
#convert to series
lineage_call_series = pd.Series(lineage_call_dict)

#append to sample annotation 
sample_annotation.loc[:, 'lineage_call'] = lineage_call_series[sample_annotation.index]

/home/rv76/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/rv76/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [47]:
sample_annotation

,sample_id,sample_num,fastq_files,coverage,lineage_call
tag,,,,,
S0,Undetermined,0,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,25.5521,4.6.1.2
S1,TC100810,1,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,151.0760,4.9
S2,TC94815,2,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,60.5541,4.3.4.2.1
S3,TC101527,3,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,70.5301,3
S4,LB95669,4,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,82.3331,2.2.1
S6,MG90097,6,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,23.7142,2.2
S7,TC101848,7,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,92.6566,4.6.1.2
S8,SM94290,8,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,72.2888,1.1.2
S9,GLK101604,9,/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_c...,132.7089,4.4.1.1


#### Save filtered Sample Annotation file

In [48]:
sample_annotation.to_csv('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/annotation CSV files/sample_annotation.csv')

############################################################################################################################################################################################################################

# [2] Create the Pipeline

############################################################################################################################################################################################################################

In [60]:
%matplotlib inline

import vcf
import os
import pandas as pd
import numpy as np
from itertools import compress
import ast
import itertools
import time
import sys
import pickle
import shutil

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq

In [61]:
#annotation file for all isolates and corresponding tags
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/annotation CSV files/sample_annotation.csv' , sep = ',').set_index('tag')

#each sub-directory has the pilon-outputed VCF file, lineage call and Qualimap report
parent_output_dir = '/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/JankyPipe/output/'

In [62]:
sample_annotation.loc[: , ['sample_id' , 'coverage' , 'lineage_call']]

,sample_id,coverage,lineage_call
tag,,,
S0,Undetermined,25.5521,4.6.1.2
S1,TC100810,151.0760,4.9
S2,TC94815,60.5541,4.3.4.2.1
S3,TC101527,70.5301,3
S4,LB95669,82.3331,2.2.1
S6,MG90097,23.7142,2.2
S7,TC101848,92.6566,4.6.1.2
S8,SM94290,72.2888,1.1.2
S9,GLK101604,132.7089,4.4.1.1


## Function to extract SNPs

In [65]:
def Get_Base_Calls_from_VCF(sample_tag):

    '''
    This function takes as input, the isolate tag, then extracts SINGLE base calls from
    the VCF file generated by Pilon. The function returns a DataFrame for a single isolate
    that contains [1] PASS calls that support the reference allele, [2] PASS calls that support
    an alternate allele and [3] AMB calls that support both the reference and alternate alleles.
    Note: Many base calls won't have an Alternate Allele.
    '''

    #directory where VCF file for sequenced isolated is stored & filename
    parent_output_dir = '/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/JankyPipe/output/'
    VCF_file = parent_output_dir + sample_tag + '/pilon/' + sample_tag + '.vcf'

    vcf_reader = vcf.Reader(open(VCF_file , 'r'))

    #position-base dictionaries
    base_order = {0:'A' , 1:'C' , 2:'G', 3:'T'} #order of output in Pilon
    base_order_r = {'A':0 , 'C':1 , 'G':2 , 'T':3}

    #create dictionaries to store information for each call
    ref_bases = {}
    qry_bases = {}
    ref_positions = {}
    quality_scores = {}
    SNP_type = {}
    PASS_filter = {}
    INFO_for_call = {}
    alt_allele_freq = {}
    valid_depth = {}

    #store the DEPTH of all base & variant calls (depth not stored for major structural variants)
    coverage_for_all_base_and_small_indel_calls = []

    #indexer for dataframe containing SNPs
    index = 0

    #iterate through each record from VCF file
    for record in vcf_reader:

        ######## store the DEPTH of all base & variant calls (depth not stored for major structural variants) ########
        try:
            coverage_for_all_base_and_small_indel_calls.append( record.INFO['DP'] )

        except KeyError: #major structural variant with no depth value
            pass 

        ######## Classify SNP-type according to Pilon Flags ########

        ### Filtering Criteria
        #---> Check that Filter is either PASS ([]) or AMBIGUOUS (['Amb']) , no 'DEL' or 'LOWCOV' flags from Pilon

        ### Call supporting REFERENCE
        #if there's no alternate allele and Filter is PASS; PASS call supporting Ref allele
        if (len(record.ALT) == 1) and (record.ALT == [None]) and (record.FILTER == []):

            alt_allele = 'Z' #no alternate allele supported by reads (might change through filtering process further down if small fraction of reads support an alternate)
            ref_allele = record.REF #allele on H37Rv
            SNP_type_for_Call = 'Ref_PASS'

        ### Call supporting ALTERNATE 
        #if there's 1 alternate allele, and Filter is PASS; PASS call supporting Alt allele
        elif (len(record.ALT) == 1) and (record.ALT != [None]) and (record.FILTER == []):

            alt_allele = str( record.ALT[0] ) #alternate allele supported by reads
            ref_allele = record.REF #allele on H37Rv
            SNP_type_for_Call = 'Alt_PASS'

        ### Call ambiguous between REFERNENCE and ALTERNATE
        #if there's 1 alternate allele, and Filter is AMB; AMB call supporting supporting neither Alt nor Ref
        elif (len(record.ALT) == 1) and (record.ALT != [None]) and (record.FILTER == ['Amb']):

            alt_allele = str( record.ALT[0] ) #alternate allele supported by reads
            ref_allele = record.REF #allele on H37Rv
            SNP_type_for_Call = 'Amb'

        ### Bad Quality Call, don't store information for this Base Call, move onto next
        else:

            continue

        ### Filtering Criteria (for Reference Confirmation or SNP)
        #---> Make sure that the length of all any alleles is 1 (No Variant or 1 Base) ; if variant, then variant is a SNP and not an INDEL
        if (len(ref_allele) == 1) and (len(alt_allele) == 1):

            ######## Retrieve Relevant information for filtering quality of Base Call ########

            # Mean Base Quality @ locus
            BQ = record.INFO['BQ']

            # Mean Mapping Quality @ locus
            MQ = record.INFO['MQ']

            # Number of Reads w/ Deletion 
            DC = record.INFO['DC']

            # Number of Reads w/ Insertion
            IC = record.INFO['IC']  
            
            # Depth of Valid Reads in Pileup
            VD = record.INFO['DP']

            ### Filtering Criteria
            #---> Mean Base Quality > 20
            #---> Mean Mapping Quality > 30
            #---> Reads Supporting Insertions <= 0.10xDepth (less than 10% of reads have an insertion)
            #---> Reads Supporting Deletions <= 0.10xDepth (less than 10% of the reads have a deletion)
            #---> Number of High Quality Reads >= 10
            if (BQ > 20) and (MQ > 30) and (DC <= VD*0.10) and (IC <= VD*0.10) and (VD >= 10):

                # Quality-Percentage - Percentage of As, Cs, Gs, Ts weighted by Q & MQ at locus
                QP = record.INFO['QP']

                ### Filtering Criteria
                #---> Make sure that Reads support at most 2 alleles (either confirm the Reference or provide evidence for an alternate allele)
                #-----> If there are 2 alleles (make sure 1 of them is the Reference Allele)

                if (sum(np.array(QP) == 0) == 3): #1 Allele supported by Reads (either Reference or an Alternate), store information for this Base Call (3 bases have NO reads supporting)

                    #create a Boolean list for the Bases that have at least 1% of support from reads
                    supported_bases_boolean = list( np.array( QP ) > 0 )

                    #positions of Bases [A,C,G,T] with at least 1% of support from reads
                    supported_bases_i = [i for i,x in enumerate(supported_bases_boolean) if x == True] #returns list of length 
                    supported_base = base_order[supported_bases_i[0]]

                    #if the Reference Allele is supported
                    if supported_base == ref_allele:
                        alt_allele_frequency = 0.0 #no alternate allele

                    #if an Alternate Allele is supported:
                    if supported_base != ref_allele:
                        alt_allele_frequency = 1.0 #alternate allele

                elif (sum(np.array(QP) == 0) == 2): #2 Alleles supported, make sure 1 is the Reference & confirm the Alternate (2 bases have NO reads supporting)

                    #position of the Reference Allele in [A,C,G,T]
                    ref_allele_base_i = base_order_r[ref_allele] 

                    #create a Boolean list for the Bases that have at least 1% of support from reads
                    supported_bases_boolean = list( np.array( QP ) > 0 )

                    #positions of Bases [A,C,G,T] with at least 1% of support from reads
                    supported_bases_i = [i for i,x in enumerate(supported_bases_boolean) if x == True] #returns list of length 2

                    #check that Reference Allele is supported by at least 1% of the reads
                    if ref_allele_base_i in supported_bases_i:

                        #find the Alternate Allele that was supported by at least 1% of the reads
                        for supported_base_i in supported_bases_i:

                            if supported_base_i != ref_allele_base_i: #then must be alternate allele

                                alt_allele = base_order[supported_base_i] #Alternate Allele
                                alt_allele_frequency = float(QP[supported_base_i]) / 100.0 #weighted percentage of reads supporting Alternate Allele
                                break
                    
                    #Reference Allele is NOT supported by at least 1% of the reads (2 alleles supported but neither is the reference), don't store information for this Base Call
                    else:
                        continue

                #more than 2 alleles supported by reads, don't store information for this Base Call
                else: 
                    continue
                    
                #only store this base call if AF >= 20%
                if alt_allele_frequency >= 0.20:

                    #After extensive filtering and categorization, store all of the pertinent information about the Base Call
                    ref_bases[index] = ref_allele
                    qry_bases[index] = alt_allele
                    ref_positions[index] = record.POS
                    PASS_filter[index] = record.FILTER
                    quality_scores[index] = record.QUAL
                    SNP_type[index] = SNP_type_for_Call
                    INFO_for_call[index] = record.INFO
                    alt_allele_freq[index] = alt_allele_frequency
                    valid_depth[index] = record.INFO['DP']

                    index += 1

    #convert dictionaries to series
    ref_bases = pd.Series(ref_bases)
    qry_bases = pd.Series(qry_bases)
    ref_positions = pd.Series(ref_positions)
    quality_scores = pd.Series(quality_scores)
    SNP_type = pd.Series(SNP_type)
    PASS_filter = pd.Series(PASS_filter)
    INFO_for_call = pd.Series(INFO_for_call)
    alt_allele_freq = pd.Series(alt_allele_freq)
    valid_depth = pd.Series(valid_depth)
    tag_series = pd.Series([sample_tag]*len(ref_bases.index) , index = ref_bases.index) #psuedo-series to store the tag id of the isolate

    #create DataFrame to hold all base calls for a given sample
    Variant_Call_DF = pd.DataFrame()
    Variant_Call_DF['ref_base'] = ref_bases
    Variant_Call_DF['alt_base'] = qry_bases
    Variant_Call_DF['ref_position'] = ref_positions
    Variant_Call_DF['quality'] = quality_scores
    Variant_Call_DF['SNP_type'] = SNP_type
    Variant_Call_DF['PASS_filter'] = PASS_filter
    Variant_Call_DF['INFO'] = INFO_for_call
    Variant_Call_DF['alt_AF'] = alt_allele_freq
    Variant_Call_DF['depth'] = valid_depth
    Variant_Call_DF['tag'] = tag_series

    ### Filtering Criteria (high coverage)
    #---> Filter out Base Calls with total # of reads aligning above the 99th percentile of covered positions in sample
    ninety_ninth_percentile_coverage = np.percentile( np.array( coverage_for_all_base_and_small_indel_calls ) , 99 )
    Variant_Call_DF = Variant_Call_DF[Variant_Call_DF.depth < ninety_ninth_percentile_coverage] #keep base calls that dont have unusually high coverage

    #reset index after dropping base calls with unusually high coverage
    Variant_Call_DF.reset_index(inplace = True , drop = True)
    
    #save single base call variants as a pickle
    Variant_Call_DF.to_pickle(parent_output_dir + sample_tag +  '/' + sample_tag + '_SNP_calls.pkl')

Call function

In [ ]:
## Get_Base_Calls_from_VCF('S0')

In [66]:
for sample_tag in list(sample_annotation.index):

    Get_Base_Calls_from_VCF(sample_tag)
    print sample_tag

S0
S1
S2
S3
S4
S6
S7
S8
S9
S10
S11
S12
S13
S14
S15
S16
S18
S19
S20
S21
S22
S23
S24
S25


############################################################################################################################################################################################################################

# Submit Jobs

############################################################################################################################################################################################################################

In [7]:
import os
import pandas as pd
from slurmpy import Slurm
import numpy as np

#### Import annotation file for all Samples

In [8]:
#annotation file for all isolates and corresponding tags
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/annotation CSV files/sample_annotation.csv' , sep = ',').set_index('tag')

In [9]:
sample_annotation.head()

,fastq_files,population,run_ID,sample_ID,sample_order,tag,isolate_type
patient_id,,,,,,,
P000183,/n/data1/hms/dbmi/farhat/fastq_db/walker/ERR03...,WALKER,ERR039337,NaN,1,ERR039337,longitudinal
P000183,/n/data1/hms/dbmi/farhat/fastq_db/walker/ERR03...,WALKER,ERR039338,NaN,2,ERR039338,longitudinal
1960,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MMJA00000000,Peru5115,1,Peru5115,longitudinal
1960,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MMPC00000000,Peru4668,2,Peru4668,longitudinal
2491,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MMZT00000000,Peru4903,1,Peru4903,longitudinal


In [10]:
np.shape(sample_annotation)

(400, 7)

#### Submit a job for each sample to get the SNPs

In [62]:
for sample_i in range(0 , np.shape(sample_annotation)[0]):
    
    sample_i_info = sample_annotation.iloc[sample_i , :]
    
    within_host_variants_job = 'python /n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/SNPs_from_VCF.py ' + sample_i_info.tag + ' ' + sample_i_info.population + ' ' + sample_i_info.name + ' ' + str(sample_i_info.sample_order)

    #directory where you want output + error files
    os.chdir('/n/data1/hms/dbmi/farhat/Roger/dennis_mujuni_collaboration/O2_SLURM_logs/SNPs_from_VCF_files/')

    job_name = str(sample_i)

    s = Slurm(job_name , {'partition':'short' , 'n':'1' , 't':'0-00:30:00' , 'mem-per-cpu':'4G' , 'mail-type':'FAIL' , 'mail-user':'roger_vargas@g.harvard.edu'})

    #submits the job
    job_id = s.run(within_host_variants_job)

    print job_name  + ' : ' +  str(job_id)

submitted: Submitted batch job 64349498
submitted: Submitted batch job 64349499
submitted: Submitted batch job 64349500


0 : 64349498
1 : 64349499
2 : 64349500


submitted: Submitted batch job 64349501
submitted: Submitted batch job 64349502
submitted: Submitted batch job 64349503


3 : 64349501
4 : 64349502
5 : 64349503


submitted: Submitted batch job 64349504
submitted: Submitted batch job 64349505
submitted: Submitted batch job 64349506


...

submitted: Submitted batch job 64349885
submitted: Submitted batch job 64349886


382 : 64349885
383 : 64349886


submitted: Submitted batch job 64349887
submitted: Submitted batch job 64349888
submitted: Submitted batch job 64349889


384 : 64349887
385 : 64349888
386 : 64349889
387 : 64349890
388 : 64349891
389 : 64349892


submitted: Submitted batch job 64349890
submitted: Submitted batch job 64349891
submitted: Submitted batch job 64349892
